In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
train=pd.read_csv('settrain.csv')
train_labels=pd.read_csv('settrainlabels.csv')
test=pd.read_csv('settest.csv')

In [3]:
x = np.array(train.iloc[:100000,:])
y = np.array(train_labels[:100000])

In [4]:
x_test = np.array(train.iloc[100000:200000,:])
y_test = np.array(train_labels[100000:200000])

In [ ]:
model = RandomForestRegressor(n_estimators=15,criterion="mae",max_depth=10)
model.fit(x,y)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [5]:
from sklearn.metrics import accuracy_score,mean_squared_error

In [10]:
##precision de score de entrenamiento
score = model.score(x, y)
print("accuracy=%.2f%%" % ( score * 100))

accuracy=-9.21%


In [11]:
##precision en la predicción
preds = model.predict(x_test)
scores_regr = mean_squared_error(y_test, preds.round())
print("accuracy=%.2f%%" % ( scores_regr * 100)) 


accuracy=42.21%


In [12]:
y_train = model.predict(x)
scores_regr = mean_squared_error(y, y_train.round())
print("accuracy=%.2f%%" % ( scores_regr * 100)) 

accuracy=42.28%


In [24]:
preds_set = model.predict(np.array(train.iloc[:200000,:]))
scores_regr = mean_squared_error(np.array(train_labels[:200000]),preds_set.round())
print("accuracy=%.2f%%" % ( scores_regr * 100)) 

accuracy=42.24%


In [6]:
submit_format = pd.read_csv('submission_format.csv', index_col='building_id')

In [7]:
from sklearn.model_selection import KFold
import xgboost as xgb

/usr/local/lib/python3.5/dist-packages/xgboost/__init__.py:29: FutureWarning: Python 3.5 support is deprecated; XGBoost will require Python 3.6+ in the near future. Consider upgrading to Python 3.6+.
  FutureWarning)


In [17]:
kf = KFold(n_splits=3)
for train_index, test_index in kf.split(x, y):
    xgb_model = xgb.XGBRFRegressor(random_state=42).fit(
    x[train_index], y[train_index])
    
    xgb_model.predict(x[test_index])
    score = xgb_model.score(x[test_index], y[test_index])
    print("accuracy=%.2f%%" % ( score * 100)) 

accuracy=31.66%
accuracy=31.50%
accuracy=30.61%


In [19]:
preds_xgb = xgb_model.predict(x_test)

In [23]:
scores_regr = mean_squared_error(y_test,preds_xgb.round())
print("accuracy=%.2f%%" % ( scores_regr * 100)) 

accuracy=34.55%


In [29]:
testpreds_xgb = xgb_model.predict(np.array(test))

In [30]:
submission1= pd.DataFrame(
    data=testpreds_xgb,
    columns=submit_format.columns,
    index=submit_format.index
)
submission1.head()

,damage_grade
building_id,
300051,2.812683
99355,2.248150
890251,1.956883
745817,1.526289
421793,2.853945


In [33]:
submission1['damage_grade'] = submission1['damage_grade'].apply(np.round)
submission1['damage_grade'] = submission1['damage_grade'].astype('int64')

In [36]:
submission1.to_csv('submitXGBRFRegressorKF.csv') #Submit 0.6556

In [37]:
testpreds_rfr = model.predict(np.array(test))
submission2= pd.DataFrame(
    data=testpreds_rfr,
    columns=submit_format.columns,
    index=submit_format.index
)
submission2.head()

,damage_grade
building_id,
300051,2.066667
99355,2.000000
890251,2.000000
745817,2.000000
421793,2.066667


In [38]:
submission2['damage_grade'] = submission2['damage_grade'].apply(np.round)
submission2['damage_grade'] = submission2['damage_grade'].astype('int64')

In [43]:
submission1.to_csv('submitRFRegressor4.csv') #No se subió - Mismo archivo que el anterior, se modifica